# Add Dependencies

In [1]:
!pip install pypdf
!pip install wget
!pip install PyPDF2
!pip install tiktoken
!pip install openai
!pip install plotly
!pip install requests

In [2]:
import sys
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib import patches
import argparse
from pypdf import PdfReader
from pathlib import Path
import requests
import pandas as pd
import numpy as np
import openai 
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity
import requests

# Add API Key

In [31]:
openai.api_key = "_____"

# Get Paper From Arxiv

In [4]:
arxiv_url = "https://arxiv.org/pdf/1706.03762.pdf"
paper_name = "1706.03762.pdf"
response = requests.get(arxiv_url)
with open(paper_name, "wb") as f:
    f.write(response.content)

# Parse PDF

In [5]:
def parse_paper(path):
    reader = PdfReader(path)
    number_of_pages = len(reader.pages)
    paper_text = []
    for i in range(number_of_pages):
        page = reader.pages[i]
        page_text = []

        def visitor_body(text, cm, tm, fontDict, fontSize):
            x = tm[4]
            y = tm[5]
            if (y > 50 and y < 720) and (len(text.strip()) > 1):
                page_text.append({
                    'fontsize': fontSize,
                    'text': text.strip().replace('\x03', ''),
                    'x': x,
                    'y': y
                })

        _ = page.extract_text(visitor_text=visitor_body)

        blob_font_size = None
        blob_text = ''
        processed_text = []

        for t in page_text:
            if t['fontsize'] == blob_font_size:
                blob_text += f" {t['text']}"
            else:
                if blob_font_size is not None and len(blob_text) > 1:
                    processed_text.append({
                        'fontsize': blob_font_size,
                        'text': blob_text,
                        'page': i
                    })
                blob_font_size = t['fontsize']
                blob_text = t['text']
        paper_text += processed_text
    return paper_text


In [6]:
paper_text = parse_paper("papers/1706.03762.pdf")

# Filter Small Rows

In [25]:
filtered_paper_text = []
for row in paper_text:
    if len(row['text']) < 50:
        continue
    filtered_paper_text.append(row)

# Convert Paper Text To DataFrame

In [26]:
df = pd.DataFrame(filtered_paper_text)
# df.head()

for index, row in df.iterrows() :
    print(row["text"])
    print()

Ashish Vaswani Google Brain avaswani@google.com Noam Shazeer Google Brain noam@google.com Niki Parmar Google Research nikip@google.com Jakob Uszkoreit Google Research usz@google.com Llion Jones Google Research llion@google.com Aidan N. Gomez

University of Toronto aidan@cs.toronto.edu Łukasz Kaiser Google Brain lukaszkaiser@google.com Illia Polosukhin

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring signiﬁcantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English- to-German translation task,

# Get Embeddings

In [27]:
embedding_model = "text-embedding-ada-002"
embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
df["embeddings"] = embeddings

# Search Embeddings

In [28]:
def search_embeddings(df, query, n=3, pprint=True):
    query_embedding = get_embedding(
        query,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        
    )
    return results

# Ask Questions

In [30]:
results = search_embeddings(df, "what model is introduced in this paper?", n=3)
results.iloc[0]['text']

'Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started the effort to evaluate this idea. Ashish, with Illia, designed and implemented the ﬁrst Transformer models and has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head attention and the parameter-free position representation and became the other person involved in nearly every detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and efﬁcient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating our research. Work performed while at Google Brain. Work performed while at Google Research. 31st Conference o